# Inflation Insights Final Project

Group Members: Sam Pham, Abbad Chamma, Miguel Hernandez, Logan May

## Introduction

Inflation plays a significant role in financial stability and the purchasing power of consumers, influencing everything from daily expenses to long-term economic planning. As inflation fluctuates across global economies, understanding how economic indicators affect inflation is crucial for evaluating economic health. The main problem we want to address is determining how inflation impacts consumer purchasing power and identifying which economic factors influence inflation the most across various countries. To investigate this, we plan to analyze global inflation trends and examine how they relate to key financial indicators. We will do this by using regression and clustering models/techniques to find patterns on how inflation affects consumer purchasing power and identify which economic environments are more vulnerable to inflationary pressure.


## The Data

The data we will use in this project comes from an existing Kaggle dataset called "Global Economic Indicators (2010–2025)- World bank" by Tanishk Sharma. This dataset has 16 key features including: Country Name, Country Id, Year, Inflation (Consumer Price Index %), GDP (Current USD), GDP per Capita, GDP Growth (% Annual), Unemployment Rate, Real Interest Rate, Public Debt (% of GDP), Government Expense and Revenue, Current Account Balance, Gross National Income, and Tax Revenue. These features provide a comprehensive view of the economic landscape across various countries, which makes this dataset well-suited for our problem.

## Data Preprocessing


## Data Understanding and Visualization

### Set the style

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style
sns.set(style="whitegrid")


In [ ]:
# Load the preprocessed data
df = pd.read_csv('preprocessed_data/world_bank_data_cleaned.csv')


### Correlation Heatmap
th

In [ ]:
plt.figure(figsize=(12, 10))
corr = df.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of Economic Indicators")
plt.show()


### Global Inflation Trend



In [ ]:
plt.figure(figsize=(12, 6))
# Group by year to get the global average
global_trends = df.groupby('year')[['Inflation (CPI %)', 'GDP Growth (% Annual)']].mean()
sns.lineplot(data=global_trends)
plt.title("Global Average Inflation vs GDP Growth (2010-2023)")
plt.ylabel("Percentage")
plt.show()


### Scatter Plot: Inflation vs Unemployment


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Unemployment Rate (%)', y='Inflation (CPI %)', alpha=0.5)
plt.title("Inflation vs. Unemployment (Phillips Curve Check)")
plt.ylim(-10, 50)  # Zoom in to ignore extreme hyperinflation outliers
plt.show()


### Distribution of Inflation


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['Inflation (CPI %)'], bins=50, kde=True)
plt.title("Distribution of Inflation (Right-Skewed)")
plt.xlim(-10, 60)  # Limit x-axis to see the main bulk of data
plt.show()


## Linear Regression

## Clustering


## Evaluation


## Impact


## Conclusion

## References

[Project Files](https://github.com/spham64/inflation-insights)

[Kaggle Dataset](https://www.kaggle.com/datasets/tanishksharma9905/global-economic-indicators-20102025)